In [1]:
# Install openpyxl if needed
!pip install openpyxl

import pandas as pd

# Load dataset
df = pd.read_excel("/content/Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
# Drop rows without CustomerID
df.dropna(subset=['CustomerID'], inplace=True)

# Remove cancelled orders (those starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove rows with negative quantity
df = df[df['Quantity'] > 0]

# Create a TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Convert InvoiceDate to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [3]:
# Save full cleaned dataset
df.to_csv("ecommerce_cleaned.csv", index=False)

In [4]:
# Create Month column
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

# Group by month and calculate revenue
monthly_sales = df.groupby('InvoiceMonth')['TotalPrice'].sum().reset_index()

# Convert period to datetime for plotting/modeling
monthly_sales['InvoiceMonth'] = monthly_sales['InvoiceMonth'].dt.to_timestamp()

# Save for future use (Tableau or Prophet)
monthly_sales.to_csv("monthly_sales.csv", index=False)

monthly_sales.head()

,InvoiceMonth,TotalPrice
0,2010-12-01,572713.890
1,2011-01-01,569445.040
2,2011-02-01,447137.350
3,2011-03-01,595500.760
4,2011-04-01,469200.361


In [5]:
print("Earliest date:", df['InvoiceDate'].min())
print("Latest date:", df['InvoiceDate'].max())

# Count unique months
print("Number of unique months:", df['InvoiceDate'].dt.to_period('M').nunique())

Earliest date: 2010-12-01 08:26:00
Latest date: 2011-12-09 12:50:00
Number of unique months: 13


In [6]:
monthly_sales = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum().reset_index()
monthly_sales['InvoiceMonth'] = monthly_sales['InvoiceDate'].dt.to_timestamp()
monthly_sales = monthly_sales[['InvoiceMonth', 'TotalPrice']]
monthly_sales.to_csv("monthly_sales.csv", index=False)

print(monthly_sales)

   InvoiceMonth   TotalPrice
0    2010-12-01   572713.890
1    2011-01-01   569445.040
2    2011-02-01   447137.350
3    2011-03-01   595500.760
4    2011-04-01   469200.361
5    2011-05-01   678594.560
6    2011-06-01   661213.690
7    2011-07-01   600091.011
8    2011-08-01   645343.900
9    2011-09-01   952838.382
10   2011-10-01  1039318.790
11   2011-11-01  1161817.380
12   2011-12-01   518192.790


In [7]:
from google.colab import files
files.download("monthly_sales.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip install prophet

In [9]:
import pandas as pd

# Load the sales data
df = pd.read_csv("monthly_sales.csv")
df.columns = ['ds', 'y']  # Prophet requires 'ds' = date, 'y' = value
df['ds'] = pd.to_datetime(df['ds'])

df.head()

,ds,y
0,2010-12-01,572713.890
1,2011-01-01,569445.040
2,2011-02-01,447137.350
3,2011-03-01,595500.760
4,2011-04-01,469200.361


In [10]:
from prophet import Prophet

model = Prophet()
model.fit(df)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 9.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwxzx7hul/j5k66l3m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwxzx7hul/ucwzzdc3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38953', 'data', 'file=/tmp/tmpwxzx7hul/j5k66l3m.json', 'init=/tmp/tmpwxzx7hul/ucwzzdc3.json', 'output', 'file=/tmp/tmpwxzx7hul/prophet_modelha11lkfm/prophet_model-20250615010204.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
01:02:05 - cmdstanpy - INFO - Chain [1] start processing
IN

In [11]:
future = model.make_future_dataframe(periods=6, freq='M')  # Forecast next 6 months
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


,ds,yhat,yhat_lower,yhat_upper
14,2012-01-31,9.679451e+05,746676.117759,1.200624e+06
15,2012-02-29,1.001596e+06,775090.799492,1.219773e+06
16,2012-03-31,1.037569e+06,828156.996007,1.254157e+06
17,2012-04-30,1.072380e+06,856590.427670,1.294348e+06
18,2012-05-31,1.108352e+06,877332.759520,1.319251e+06


In [12]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)  # Interactive forecast chart

In [13]:
forecast_df = forecast[['ds', 'yhat']]
forecast_df.to_csv("sales_forecast.csv", index=False)

In [14]:
import pandas as pd

# Load the original Excel file again
df = pd.read_excel("/content/Online Retail.xlsx")

# Clean it again
df.dropna(subset=['CustomerID'], inplace=True)
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[df['Quantity'] > 0]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [15]:
# Save as cleaned CSV
df.to_csv("ecommerce_cleaned.csv", index=False)

# Download the file
from google.colab import files
files.download("ecommerce_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# Extract date and forecasted values
forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Save to CSV
forecast_df.to_csv("sales_forecast.csv", index=False)

In [17]:
from google.colab import files
files.download("sales_forecast.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>